# N Pulse Generator

Configure a repeating burst of 2–4 identical pulses (limited by PIO instruction memory) using the same knobs as the double pulse generator. Set `pulse_width_ns`, `separation_ns`, `repetition_rate_hz`, and the desired `num_pulses`.


## Imports

The examples below use the serial command sender, but the `WebCommandSender` can be substituted if you start the on-board webserver.

In [19]:
import sys
import os
import time

notebook_dir = os.getcwd()
sys.path.append(os.path.abspath(os.path.join(notebook_dir, '..', 'source')))

from serial_controller import SerialController
from serial_command_sender import SerialCommandSender
from web_controller import WebController
from web_command_sender import WebCommandSender


## Configure the n pulse generator

Set `SERIAL_PORT`, `hardware_id`, and the `settings` dictionary below. Running it sends `list_commands` to verify connectivity before creating hardware.

In [20]:
SERIAL_PORT = "/dev/ttyACM0"  # Replace with the Pico W serial port on your system
hardware_id = "n_pulse_gen"

settings = {
    'pin_number': 15,
    'num_pulses': 3,
    'pulse_width_ns': 500,
    'separation_ns': 2000,
    'repetition_rate_hz': 10000,
    'start_on_init': False
}

serial_controller = SerialController(port=SERIAL_PORT, debug=False)
serial_controller.connect()
serial_command_sender = SerialCommandSender(serial_controller)

try:
    commands = serial_command_sender.send_command('list_commands')
    print('Available commands:' + commands)
finally:
    serial_controller.close()


Connected to /dev/ttyACM0
Available commands:No valid response found or invalid format.
Connection to /dev/ttyACM0 closed.


## Create and tune the n pulse waveform

After the quick command test succeeds, run this to create the `n_pulse` hardware and apply live updates. Adjust `separation_ns`, `num_pulses`, or `repetition_rate_hz` just like you would with the double pulse generator.


In [ ]:
serial_controller = SerialController(port=SERIAL_PORT, debug=False)
serial_controller.connect()
serial_command_sender = SerialCommandSender(serial_controller)

try:
    print(serial_command_sender.send_command('create', 'n_pulse', settings, hardware_id))
    print(serial_command_sender.send_command('start', hardware_id))

    time.sleep(5)

    # Try a wider separation and more pulses
    response = serial_command_sender.send_command(
        'apply_hardware_settings', hardware_id, {
            'num_pulses': 4,
            'pulse_width_ns': 400,
            'separation_ns': 2500,
            'repetition_rate_hz': 7000,
        }
    )
    print('Updated burst ->', response)
    time.sleep(5)

    # Sweep the repetition rate similar to the double pulse demo
    for rate in (5000, 12000):
        response = serial_command_sender.send_command(
            'apply_hardware_settings', hardware_id, {
                'repetition_rate_hz': rate,
            }
        )
        print(f'Repetition rate set to {rate} Hz ->', response)
        time.sleep(1)

    time.sleep(2)
    print(serial_command_sender.send_command('stop', hardware_id))
    print(serial_command_sender.send_command('delete', hardware_id))
finally:
    serial_controller.close()


Connected to /dev/ttyACM0
Created n pulse hardware with ID n_pulse_gen.
Hardware with ID n_pulse_gen started.
Updated burst -> No valid response found or invalid format.
Repetition rate set to 5000 Hz -> Settings applied to hardware ID n_pulse_gen: {'repetition_rate_hz': 5000}
Repetition rate set to 12000 Hz -> Settings applied to hardware ID n_pulse_gen: {'repetition_rate_hz': 12000}
Component on hardware ID n_pulse_gen stopped.
Hardware with ID n_pulse_gen deleted.
Connection to /dev/ttyACM0 closed.


### Wi-Fi control

To use Wi-Fi instead of serial, start the webserver on the Pico, create a `WebController(base_url='http://<ip>:<port>', debug=True)` and pass it to `WebCommandSender`. Command usage stays the same, e.g. `web_command_sender.send_command('create', 'n_pulse', settings, hardware_id)`.
